In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


restaurant_df= pd.read_csv("Dataset.csv")


restaurant_df['Cuisines'] = restaurant_df['Cuisines'].fillna('Unknown')
restaurant_df['Average Cost for two'] = restaurant_df['Average Cost for two'].fillna(restaurant_df['Average Cost for two'].median())
restaurant_df['Aggregate rating'] = restaurant_df['Aggregate rating'].fillna(restaurant_df['Aggregate rating'].mean())


restaurant_df['Has Table booking'] =restaurant_df['Has Table booking'].apply(lambda x: 1 if x == 'Yes' else 0)
restaurant_df['Has Online delivery'] = restaurant_df['Has Online delivery'].apply(lambda x: 1 if x == 'Yes' else 0)
restaurant_df['Is delivering now'] = restaurant_df['Is delivering now'].apply(lambda x: 1 if x == 'Yes' else 0)


restaurant_df['Combined Features'] = (
    restaurant_df['Cuisines'] + " " +
    restaurant_df['City'] + " " +
    restaurant_df['Locality'] + " " +
    restaurant_df['Price range'].astype(str) + " " +
    restaurant_df['Aggregate rating'].astype(str)
)


vectorizer = CountVectorizer()
feature_matrix = vectorizer.fit_transform(restaurant_df['Combined Features'])


similarity_matrix = cosine_similarity(feature_matrix)


def recommend_restaurants(preferences, top_n=10):
    
    pref_features = (
        preferences['Cuisines'] + " " +
        preferences['City'] + " " +
        preferences['Locality'] + " " +
        str(preferences['Price range']) + " " +
        str(preferences['Minimum Rating'])
    )
    pref_vector = vectorizer.transform([pref_features])

   
    scores = cosine_similarity(pref_vector, feature_matrix).flatten()

    
    top_indices = scores.argsort()[-top_n:][::-1]

    
    recommended_data = restaurant_df.iloc[top_indices]

    
    filtered_data = recommended_data[
        (recommended_data['Price range'] <= preferences['Price range']) &
        (recommended_data['Aggregate rating'] >= preferences['Minimum Rating'])
    ]

    if filtered_data.empty:
        return recommended_data[['Restaurant Name', 'Cuisines', 'City', 'Locality', 'Average Cost for two', 'Aggregate rating']].head(top_n)
    
    return filtered_data[['Restaurant Name', 'Cuisines', 'City', 'Locality', 'Average Cost for two', 'Aggregate rating']].head(top_n)

user_preferences = {
    'Cuisines': 'Italian',
    'City': 'New York',
    'Locality': 'Manhattan',
    'Price range': 3,
    'Minimum Rating': 4.0
}

recommendations = recommend_restaurants(user_preferences, top_n=5)
print("Recommended Restaurants:")
print(recommendations)


Recommended Restaurants:
         Restaurant Name                   Cuisines       City  \
5931  Pizza Hut Delivery  Italian, Pizza, Fast Food  New Delhi   
6057        Cafe Brownie              Cafe, Italian  New Delhi   
6984      Katwalk Lounge              Cafe, Italian  New Delhi   
4328                 Ego       Continental, Italian  New Delhi   
4304            Fresc Co     Italian, Mediterranean  New Delhi   

                Locality  Average Cost for two  Aggregate rating  
5931  New Friends Colony                   800               3.9  
6057           Paharganj                   600               0.0  
6984        Satyaniketan                   600               3.3  
4328              Jasola                   800               3.3  
4304             Janpath                  1900               3.8  


In [2]:
user_preferences = {
    'Cuisines': 'Chinese',
    'City': 'San Francisco',
    'Locality': 'Chinatown',
    'Price range': 2,
    'Minimum Rating': 3.5
}

recommendations = recommend_restaurants(user_preferences, top_n=5)
print("Recommended Restaurants:")
print(recommendations)


Recommended Restaurants:
     Restaurant Name Cuisines       City  Locality  Average Cost for two  \
2463        Hao Ming  Chinese  Mangalore  Balmatta                   600   

      Aggregate rating  
2463               3.5  
